# 灵活定义模型

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

In [4]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0591,  0.1849,  0.1661, -0.0126,  0.0925, -0.1268,  0.4089, -0.3859,
          0.0722, -0.0596],
        [ 0.0228,  0.2071,  0.0053, -0.0107, -0.0475, -0.1832,  0.0153, -0.0688,
         -0.0883, -0.0483]], grad_fn=<AddmmBackward0>)

In [12]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [42]:
net = MLP()
net(X)

tensor([[-0.1376,  0.1313, -0.1033,  0.2142,  0.2344,  0.0168, -0.0389,  0.1767,
          0.1074, -0.1541],
        [-0.0987, -0.1181,  0.1168, -0.0898,  0.1957,  0.0484,  0.0573,  0.2531,
          0.0019, -0.0516]], grad_fn=<AddmmBackward0>)

In [44]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X
    
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X) 

tensor([[-0.2607, -0.0049,  0.0585,  0.1981, -0.1583, -0.0051, -0.1115, -0.0918,
          0.1581, -0.0102],
        [-0.1411,  0.0461,  0.1302,  0.1771,  0.0656,  0.0444, -0.0250,  0.0042,
          0.1786, -0.1683]], grad_fn=<AddmmBackward0>)

In [47]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X) 

tensor(-0.3006, grad_fn=<SumBackward0>)

In [57]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.ReLU()
        )
        self.linear = nn.Linear(32, 16)
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1125, grad_fn=<SumBackward0>)

---
# 参数管理

In [59]:
import torch
from torch import nn

net = nn.Sequential(
    nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1)
)

X = torch.rand(size=(2, 4))
X, net(X)

(tensor([[0.6748, 0.0479, 0.3486, 0.1065],
         [0.2062, 0.3212, 0.1361, 0.0084]]),
 tensor([[0.3160],
         [0.3673]], grad_fn=<AddmmBackward0>))

In [61]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.2438, -0.1439,  0.1815,  0.1498, -0.0166,  0.3248,  0.0722, -0.1403]])), ('bias', tensor([0.1791]))])


In [62]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1791], requires_grad=True)
tensor([0.1791])


In [65]:
net[2].weight.grad == None

True

In [68]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])


('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [70]:
net.state_dict()['2.bias'].data

tensor([0.1791])

In [82]:
def block1():
    return nn.Sequential(
        nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU()
    )

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net
    
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.3679],
        [0.3679]], grad_fn=<AddmmBackward0>)

In [83]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [99]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0124, -0.0023,  0.0099,  0.0177]), tensor(0.))

In [103]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [120]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.5009,  0.6392, -0.2600,  0.3612])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [124]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init",
            *[(name, param.shape) for name, param in m.named_parameters()][0]
        )
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
net.apply(my_init)
net[0].weight.data[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, -9.0800,  0.0000, -0.0000],
        [-0.0000,  0.0000, -6.2521,  9.6317]])

In [131]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -2.0800,  7.0000,  7.0000])

In [132]:
shared = nn.Linear(8, 8)  # 参数绑定
net = nn.Sequential(
    nn.Linear(4, 8), nn.ReLU(), shared,
    nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1)
)

net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


---
# 自定义层

In [134]:
import torch
import torch.nn.functional as F
from torch import nn

class CentereLayer(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        return X - X.mean()
    
layer = CentereLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [136]:
net = nn.Sequential(nn.Linear(8, 128), CentereLayer())

Y = net(torch.rand(4, 8))
Y.mean()

tensor(4.1910e-09, grad_fn=<MeanBackward0>)

In [139]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

dense = MyLinear(5, 3)
dense.weight
    

Parameter containing:
tensor([[-0.2825, -0.7970, -0.9676],
        [ 0.0072,  2.7178, -0.1844],
        [ 1.0416,  0.3799, -0.9511],
        [-1.3716,  0.4229,  0.7071],
        [-1.1484, -0.0903,  0.4456]], requires_grad=True)

In [141]:
dense(torch.rand(2, 5))

tensor([[0.0000, 3.1661, 0.0000],
        [0.0000, 3.4664, 0.0000]])

In [143]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.0000],
        [3.0790]])

---
# 读写文件

In [150]:
import torch
from torch import nn
from torch.nn import functional as F
import os

os.makedirs("../data/16", exist_ok=True)

x = torch.arange(4)
torch.save(x, '../data/16/x-file')

x2 = torch.load("../data/16/x-file")

x2

tensor([0, 1, 2, 3])

In [155]:
y = torch.zeros(4)
torch.save([x, y], '../data/16/x-files')
x2, y2 = torch.load('../data/16/x-files')

(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [157]:
mydict = {'x': x, 'y': y}
torch.save(mydict, '../data/16/mydict')
mydict2 = torch.load('../data/16/mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [165]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))
    
net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)
Y

tensor([[ 0.4270, -0.0587, -0.0671, -0.3083, -0.3547, -0.2275,  0.2460, -0.3038,
         -0.4420,  0.1681],
        [ 0.8226,  0.0966, -0.4710, -0.0720, -0.0464, -0.2008, -0.1909, -0.0593,
         -0.0291,  0.2163]], grad_fn=<AddmmBackward0>)

In [168]:
torch.save(net.state_dict(), '../data/16/mlp.params')

In [170]:
clone = MLP()
clone.load_state_dict(torch.load("../data/16/mlp.params"))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [171]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])